In [15]:
from pathlib import Path
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

TRAIN_LOCAL = Path("assignment2train.csv")
TEST_LOCAL  = Path("assignment2test.csv")

TRAIN_URL = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment2/main/assignment2train.csv"
TEST_URL  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment2/main/assignment2test.csv"

def read_csv_safely(local_path: Path, url: str) -> pd.DataFrame:
    if local_path.exists():
        return pd.read_csv(local_path)
    else:
        print(f"Local file {local_path} not found, using GitHub version.")
        return pd.read_csv(url)

train = read_csv_safely(TRAIN_LOCAL, TRAIN_URL)
test  = read_csv_safely(TEST_LOCAL, TEST_URL)





Local file assignment2train.csv not found, using GitHub version.
Local file assignment2test.csv not found, using GitHub version.


In [16]:
def feature_engineer(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = [c.strip() for c in d.columns]

    if "DateTime" in d.columns:
        dt = pd.to_datetime(d["DateTime"], errors="coerce")
        d["hour"]  = dt.dt.hour
        d["dow"]   = dt.dt.dayofweek
        d["month"] = dt.dt.month

        d["is_breakfast"] = d["hour"].between(6, 10).astype(int)
        d["is_lunch"]     = d["hour"].between(11, 14).astype(int)
        d["is_afternoon"] = d["hour"].between(15, 17).astype(int)
        d["is_dinner"]    = d["hour"].between(18, 21).astype(int)
        d["is_weekend"]   = d["dow"].isin([5, 6]).astype(int)

    for col in ["Total", "Discounts"]:
        if col in d.columns:
            d[col] = pd.to_numeric(d[col], errors="coerce")

    if "Total" in d.columns:
        d["Total_capped"] = d["Total"].clip(lower=0, upper=d["Total"].quantile(0.99))
        d["is_total_gt5"] = (d["Total"] >= 5).astype(int)
        d["is_total_gt8"] = (d["Total"] >= 8).astype(int)

    if "Discounts" in d.columns:
        d["Discounts_capped"] = d["Discounts"].clip(lower=0, upper=d["Discounts"].quantile(0.99))

    for c in ["id", "DateTime"]:
        if c in d.columns:
            d.drop(columns=c, inplace=True)

    return d

train = feature_engineer(train)
test  = feature_engineer(test)
train.head()


,Total,Discounts,Brewed_Coffee_12_oz,Brewed_Coffee_16_oz,Aquafina_Water,Muffin_Pastry_Case,Extra_Syrup,Bottled_Soda_Mt_Dew,Bottled_Soda_Diet_Pepsi,Latte_16_oz,...,month,is_breakfast,is_lunch,is_afternoon,is_dinner,is_weekend,Total_capped,is_total_gt5,is_total_gt8,Discounts_capped
0,3.96,0.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,3.96,0,0,0.0
1,3.57,0.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,3.57,0,0,0.0
2,2.21,0.0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,2.21,0,0,0.0
3,2.84,0.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,2.84,0,0,0.0
4,4.58,0.0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,4.58,0,0,0.0


In [17]:
y = train["meal"].astype(int)
X = train.drop(columns=["meal"])

imputer   = SimpleImputer(strategy="most_frequent")
X_imputed = imputer.fit_transform(X)
X.shape, y.shape


((14169, 93), (14169,))

In [18]:
model = RandomForestClassifier(
    n_estimators=900,
    max_features="sqrt",
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
)
model


RandomForestClassifier(n_estimators=900, n_jobs=-1, random_state=42)

In [19]:
modelFit = model.fit(X_imputed, y)
hasattr(modelFit, "estimators_"), model is modelFit


(True, True)

In [20]:
for col in X.columns:
    if col not in test.columns:
        test[col] = 0
test = test[X.columns]

X_test = imputer.transform(test)
pred = [int(x) for x in modelFit.predict(X_test)]

pd.DataFrame({"pred": pred}).head()


,pred
0,0
1,0
2,0
3,0
4,1
